In [1]:
import pandas as pd
import pickle
import numpy as np

In [2]:
ticker = 'AAPL'
filename = '../../../Data_Store/Yah_Daily_Prices/Daily_Price_Data/{}.pk'.format(ticker)
with open(filename, 'rb') as file:
    # Change Index to timestamp for original Polygon Data
    # Change Index to Date for Yah Finance Data
    data = pd.DataFrame(pickle.load(file)).set_index('Date', drop=True)

In [3]:
print(data.tail())

                  High         Low        Open       Close       Volume  \
Date                                                                      
2021-04-23  135.119995  132.160004  132.160004  134.320007   78657500.0   
2021-04-26  135.059998  133.559998  134.830002  134.720001   66905100.0   
2021-04-27  135.410004  134.110001  135.009995  134.389999   66015800.0   
2021-04-28  135.020004  133.080002  134.309998  133.580002  106589600.0   
2021-04-29  137.070007  132.449997  136.470001  133.479996  150777100.0   

             Adj Close  
Date                    
2021-04-23  134.320007  
2021-04-26  134.720001  
2021-04-27  134.389999  
2021-04-28  133.580002  
2021-04-29  133.479996  


In [4]:
data_chunk = data[-50:]
data_chunk.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2021-02-18,130.000000,127.410004,129.199997,129.710007,96856700.0,129.710007
2021-02-19,130.710007,128.800003,130.240005,129.869995,87668800.0,129.869995
2021-02-22,129.720001,125.599998,128.009995,126.000000,103916400.0,126.000000
2021-02-23,126.709999,118.389999,123.760002,125.860001,158273000.0,125.860001
2021-02-24,125.559998,122.230003,124.940002,125.349998,111039900.0,125.349998


In [5]:
data_chunk['Pct Change'] = data_chunk['Close'].pct_change()

<ipython-input-5-b521cdf5a631>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_chunk['Pct Change'] = data_chunk['Close'].pct_change()


In [6]:
data_chunk.head()

,High,Low,Open,Close,Volume,Adj Close,Pct Change
Date,,,,,,,
2021-02-18,130.000000,127.410004,129.199997,129.710007,96856700.0,129.710007,NaN
2021-02-19,130.710007,128.800003,130.240005,129.869995,87668800.0,129.869995,0.001233
2021-02-22,129.720001,125.599998,128.009995,126.000000,103916400.0,126.000000,-0.029799
2021-02-23,126.709999,118.389999,123.760002,125.860001,158273000.0,125.860001,-0.001111
2021-02-24,125.559998,122.230003,124.940002,125.349998,111039900.0,125.349998,-0.004052


In [7]:
data_chunk = data_chunk.fillna(0)

In [8]:
data_chunk['Pos Change'] = data_chunk.apply(lambda x: x['Pct Change'] if x['Pct Change'] > 0.0 else np.nan, axis=1)

In [9]:
data_chunk['Neg Change'] = data_chunk.apply(lambda x: x['Pct Change'] if x['Pct Change'] < 0.0 else np.nan, axis=1)

In [10]:
print(data_chunk.head())

                  High         Low        Open       Close       Volume  \
Date                                                                      
2021-02-18  130.000000  127.410004  129.199997  129.710007   96856700.0   
2021-02-19  130.710007  128.800003  130.240005  129.869995   87668800.0   
2021-02-22  129.720001  125.599998  128.009995  126.000000  103916400.0   
2021-02-23  126.709999  118.389999  123.760002  125.860001  158273000.0   
2021-02-24  125.559998  122.230003  124.940002  125.349998  111039900.0   

             Adj Close  Pct Change  Pos Change  Neg Change  
Date                                                        
2021-02-18  129.710007    0.000000         NaN         NaN  
2021-02-19  129.869995    0.001233    0.001233         NaN  
2021-02-22  126.000000   -0.029799         NaN   -0.029799  
2021-02-23  125.860001   -0.001111         NaN   -0.001111  
2021-02-24  125.349998   -0.004052         NaN   -0.004052  


In [15]:
data_chunk['Pos Sum'] = (data_chunk['Pos Change'].rolling(window=14, min_periods=1).sum())/14.0
data_chunk['Neg Sum'] = -1*(data_chunk['Neg Change'].rolling(window=14, min_periods=1).sum())/14.0

In [16]:
data_chunk['RSI'] = [(100 - (100/(1+(data_chunk['Pos Sum'][i]/data_chunk['Neg Sum'][i])))) for i in range(0, len(data_chunk))]

In [17]:
print(data_chunk[-20:])

                  High         Low        Open       Close       Volume  \
Date                                                                      
2021-04-01  124.180000  122.489998  123.660004  123.000000   75089100.0   
2021-04-05  126.160004  123.070000  123.870003  125.900002   88651200.0   
2021-04-06  127.129997  125.650002  126.500000  126.209999   80171300.0   
2021-04-07  127.919998  125.139999  125.830002  127.900002   83466700.0   
2021-04-08  130.389999  128.520004  128.949997  130.360001   88844600.0   
2021-04-09  133.039993  129.470001  129.800003  133.000000  106686700.0   
2021-04-12  132.850006  130.630005  132.520004  131.240005   91420000.0   
2021-04-13  134.660004  131.929993  132.440002  134.429993   91266500.0   
2021-04-14  135.000000  131.660004  134.940002  132.029999   87222800.0   
2021-04-15  135.000000  133.639999  133.820007  134.500000   89347100.0   
2021-04-16  134.669998  133.279999  134.300003  134.160004   84922400.0   
2021-04-19  135.470001  1